In [7]:
!pip install kaggle

In [16]:
import kagglehub
import os
import shutil

# Download latest version
dataset_path = kagglehub.dataset_download("trolukovich/food5k-image-dataset")

# Define the destination folder
destination_folder = "./Food-5K"  # Or any other desired folder

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Move the downloaded dataset to the destination folder
for filename in os.listdir(dataset_path):
    source_path = os.path.join(dataset_path, filename)
    destination_path = os.path.join(destination_folder, filename)
    shutil.move(source_path, destination_path)

print("Dataset downloaded to:", destination_folder)

Dataset downloaded to: ./Food-5K


In [17]:
import shutil
import os

# Define dataset paths
dataset_path = "Food-5K/training/"  # Path to the extracted dataset
food_dir = "dataset/food/"  # Destination for food images
non_food_dir = "dataset/non_food/"  # Destination for non-food images

# Create destination directories
os.makedirs(food_dir, exist_ok=True)
os.makedirs(non_food_dir, exist_ok=True)

# Move images based on filename convention
for filename in os.listdir(dataset_path):
    src = os.path.join(dataset_path, filename)
    if filename.startswith("0"):  # Food images start with '0'
        shutil.copy(src, os.path.join(food_dir, filename)) # Use copy instead of move to keep original files
    elif filename.startswith("1"):  # Non-food images start with '1'
        shutil.copy(src, os.path.join(non_food_dir, filename)) # Use copy instead of move to keep original files

print("Dataset organized successfully!")

Dataset organized successfully!


In [15]:
import os

# Define the dataset path
dataset_path = "dataset"

# Check if the dataset directory exists
if os.path.exists(dataset_path):
    print(f"The directory '{dataset_path}' exists.")

    # Check for subdirectories and their contents
    subdirectories = ["food", "non_food"]
    for subdir in subdirectories:
        subdir_path = os.path.join(dataset_path, subdir)
        if os.path.exists(subdir_path):
            print(f"  - Subdirectory '{subdir}' exists.")
            # List files in the subdirectory
            files = os.listdir(subdir_path)
            if files:
                print(f"    - Files found in '{subdir}': {files[:5]}")  # Print first 5 filenames
            else:
                print(f"    - No files found in '{subdir}'.")
        else:
            print(f"  - Subdirectory '{subdir}' does not exist.")

else:
    print(f"The directory '{dataset_path}' does not exist.")

The directory 'dataset' exists.
  - Subdirectory 'food' exists.
    - No files found in 'food'.
  - Subdirectory 'non_food' exists.
    - Files found in 'non_food': ['food', 'non_food']


In [18]:
import os
import shutil

# Define dataset paths
dataset_path = "dataset"
food_dir = os.path.join(dataset_path, "food")
non_food_dir = os.path.join(dataset_path, "non_food")

# Move incorrectly placed folders to the correct location
incorrect_food_dir = os.path.join(non_food_dir, "food")
incorrect_non_food_dir = os.path.join(non_food_dir, "non_food")

if os.path.exists(incorrect_food_dir):
    # Move contents of incorrect_food_dir to food_dir
    for item in os.listdir(incorrect_food_dir):
        src = os.path.join(incorrect_food_dir, item)
        dst = os.path.join(food_dir, item)
        shutil.move(src, dst)
    # Remove the empty incorrect_food_dir
    os.rmdir(incorrect_food_dir)
    print("Moved files from 'non_food/food' to 'dataset/food'")

if os.path.exists(incorrect_non_food_dir):
    # Move contents of incorrect_non_food_dir to non_food_dir
    for item in os.listdir(incorrect_non_food_dir):
        src = os.path.join(incorrect_non_food_dir, item)
        dst = os.path.join(non_food_dir, item)
        shutil.move(src, dst)
    # Remove the empty incorrect_non_food_dir
    os.rmdir(incorrect_non_food_dir)
    print("Moved files from 'non_food/non_food' to 'dataset/non_food'")

print("Folder structure fixed!")

Moved files from 'non_food/food' to 'dataset/food'
Moved files from 'non_food/non_food' to 'dataset/non_food'
Folder structure fixed!


In [23]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load dataset
dataset = datasets.ImageFolder(root="dataset", transform=transform)

# Split into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print("Data loaded successfully!")


Data loaded successfully!


In [24]:
import timm
import torch.nn as nn

class FoodClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(FoodClassifier, self).__init__()
        self.model = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.model.head = nn.Linear(self.model.head.in_features, num_classes)  # Modify classifier

    def forward(self, x):
        return self.model(x)

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FoodClassifier().to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [25]:
import torch.optim as optim
import torch.nn as nn # Import the necessary module

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}")

Epoch [1/5], Loss: 0.1226
Epoch [2/5], Loss: 0.0239
Epoch [3/5], Loss: 0.0269
Epoch [4/5], Loss: 0.0290
Epoch [5/5], Loss: 0.0187


In [26]:
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Validation Accuracy: {accuracy:.2f}%')


Validation Accuracy: 98.50%


In [28]:
# Define the path to save the model
model_path = "food_classifier_vit.pth"

# Save the model state dictionary
torch.save(model.state_dict(), model_path)

print(f"Model saved to {model_path}")


Model saved to food_classifier_vit.pth


In [29]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import timm
import torch.nn as nn

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model architecture
class FoodClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(FoodClassifier, self).__init__()
        self.model = timm.create_model('vit_base_patch16_224', pretrained=False)
        self.model.head = nn.Linear(self.model.head.in_features, num_classes)  # Modify classifier

    def forward(self, x):
        return self.model(x)

# Initialize model
model = FoodClassifier().to(device)

# Load model weights
model.load_state_dict(torch.load("food_classifier_vit.pth", map_location=device))
model.eval()  # Set model to evaluation mode

print("Model loaded successfully!")


<ipython-input-29-36a137873512>:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("food_classifier_vit.pth", map_location=device))


Model loaded successfully!


In [30]:
# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalization
])


In [32]:
def classify_image(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    # Make prediction
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)

    # Class labels
    labels = ["Food", "Non-Food"]

    return labels[predicted.item()]

# Test on an image
image_path = "test-2.jpeg"  # Replace with the actual image path
result = classify_image(image_path, model)
print(f"Predicted Class: {result}")


Predicted Class: Non-Food
